# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
!pip install fast_ml
from fast_ml.model_development import train_valid_test_split
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Видим что загрузили данные по старым тарифам и тарифу ультра,Установим целевой признак 'is_ultra' запишем в target. Признаки запишем в features.

## Разбейте данные на выборки

In [3]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
display(features.head())
display(target.head())

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

In [4]:
features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(df,
                                                        target = 'is_ultra', train_size=0.6, valid_size=0.2, test_size=0.2)

## Исследуйте модели

# DecisionTreeClassifier

In [5]:
for depth in range(1, 6):
        # < создаем модель, указав max_depth=depth >
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # < обучите модель >
    model.fit(features_train, target_train)
    # < найдите предсказания на валидационной выборке >
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7465007776049767
max_depth = 2 : 0.7713841368584758
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7853810264385692
max_depth = 5 : 0.7713841368584758


Вывод: лучшая модель на глубине дерева 4. 

In [6]:
criterion = ['gini', 'entropy']
for i in criterion:
    model = DecisionTreeClassifier(random_state=12345, criterion = i,max_depth=4) # обучите модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучаем модель
    predictions = model.predict(features_valid) # получаем предсказания модели
    accuracy_score(target_valid, predictions) # посчитаем качество модели
            
    print('criterion = ', i, '\nAccuracy лучшей модели:', accuracy_score(target_valid, predictions))

criterion =  gini 
Accuracy лучшей модели: 0.7853810264385692
criterion =  entropy 
Accuracy лучшей модели: 0.7791601866251944


Вывод: лучшая модель с гиперпараметром gini.

In [7]:
splitter = ['best', 'random']
for i in splitter:
    model = DecisionTreeClassifier(random_state=12345, splitter = i,max_depth=4) # обучите модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучаем модель
    predictions = model.predict(features_valid) # получаем предсказания модели
    accuracy_score(target_valid, predictions) # посчитаем качество модели
            
    print('splitter = ', i, '\nAccuracy лучшей модели:', accuracy_score(target_valid, predictions))

splitter =  best 
Accuracy лучшей модели: 0.7853810264385692
splitter =  random 
Accuracy лучшей модели: 0.7527216174183515


Вывод: лучшая модель с гиперпараметром best.

# RandomForestClassifier

In [8]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучаем модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # получаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

    print("Accuracy наилучшей модели на валидационной выборке:", best_result) 

Accuracy наилучшей модели на валидационной выборке: 0.7465007776049767
Accuracy наилучшей модели на валидационной выборке: 0.7713841368584758
Accuracy наилучшей модели на валидационной выборке: 0.7713841368584758
Accuracy наилучшей модели на валидационной выборке: 0.7838258164852255
Accuracy наилучшей модели на валидационной выборке: 0.7838258164852255
Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314


# LogisticRegression

In [9]:
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for i in solver:
    model = LogisticRegression(random_state=12345, solver = i,max_iter = 1000)
    model.fit(features_train, target_train) # обучаем модель
    predictions = model.predict(features_valid) # получаем предсказания модели
    accuracy_score(target_valid, predictions) # посчитаем качество модели
            
    print('solver = ', i, '\nAccuracy лучшей модели:', accuracy_score(target_valid, predictions))

solver =  newton-cg 
Accuracy лучшей модели: 0.7371695178849145
solver =  lbfgs 
Accuracy лучшей модели: 0.7371695178849145
solver =  liblinear 
Accuracy лучшей модели: 0.6936236391912908
solver =  sag 
Accuracy лучшей модели: 0.6858475894245724
solver =  saga 
Accuracy лучшей модели: 0.687402799377916


Вывод: лучшая модель с гиперпараметром newton-cg.

## Проверьте модель на тестовой выборке

RandomForestClassifier лучшие показатели, берем его с глубиной 10 для проверки: 

In [12]:
model = RandomForestClassifier(random_state=12345, n_estimators=10) # обучаем модель с заданным количеством деревьев
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
result = model.score(features_test, target_test) # получаем качество модели на тестовой выборке

print("Accuracy наилучшей модели на валидационной выборке:", result)  

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566


## (бонус) Проверьте модели на адекватность

In [13]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_test, target_test)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_test)
dummy_clf.score(features_test, target_test)

0.6811819595645412

#### Вывод: константная модель выдает 68%.Наша модель показывает 78%, значит она адекватна.

Лучший Accuracy получился в тестовой выборке в RandomForestClassifier при глубине 10.

Не все гиперпараметры дают большое значение Accuracy.

Итоговый Accuracy получился больше 75%.